In [1]:
from IPython.core.display import display, HTML
display(HTML('<style>.container { width: 98% !important }</style>'))

In [2]:
from utilities import *

In [3]:
import joblib
import tensorflow as tf
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

sns.set()

In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

## Import Data

In [5]:
pin = read_pin('../Data/pin.csv')
test_pin = read_pin('../Data/testPin.csv')
ble = read_ble('../Data/ble.csv')

In [6]:
data_1 = read_data('../Data/rssi5.csv')
data_2 = read_data('../Data/rssi_2.csv')

All beacons: ['0117C55D14E4']
Selecting all
All beacons: ['0117C55D14E4']
Selecting all


In [7]:
test_1 = read_data('../Data/testRssi.csv')
test_2 = read_data('../Data/testRssi_2.csv')

All beacons: ['0117C55D14E4']
Selecting all
All beacons: ['0117C55D14E4']
Selecting all


In [8]:
data = pd.concat([data_1, data_2], axis=0)
test = pd.concat([test_1, test_2], axis=0)

## Train Validation Split

In [9]:
train, validation = train_validation_split(data)

In [10]:
print(train.shape)
print(validation.shape)

(102033, 19)
(11337, 19)


In [11]:
def linear_impute(data):
    for s in scanners:
        dx = data['X'] - ble[s]['X']
        dy = data['Y'] - ble[s]['Y']
        d = np.sqrt(np.square(dy) + np.square(dx))
        reg = joblib.load('../Models/linear_distance_models_2/{}.joblib'.format(s))
        ss = pd.Series(reg.predict(d.values.reshape((-1, 1))))
        data[s].fillna(ss, inplace=True)

def impute(data):
    rolling_win = 30

    data = data.sort_values("time")
    data_rolled = data.groupby("location").rolling(rolling_win, min_periods=1).median().reset_index()
    data_imputed = data_rolled.set_index("location").groupby("location").ffill()
    data_imputed.reset_index(inplace=True)
    data_imputed["X"] = data_imputed["location"].apply(lambda x: pin[x]["X"] if x[0] == 'V' else test_pin[x]["X"])
    data_imputed["Y"] = data_imputed["location"].apply(lambda x: pin[x]["Y"] if x[0] == 'V' else test_pin[x]["Y"])
    linear_impute(data_imputed)
    return data_imputed

def impute_ambient(data):
    rolling_win = 30

    data = data.sort_values("time")
    data_rolled = data.rolling(rolling_win, min_periods=1).median().reset_index()
    data_imputed = data_rolled.ffill()
    data_imputed['location'] = 'ambient'
    linear_impute(data_imputed)
    return data_imputed

In [12]:
train_imputed = impute(train)
validation_imputed = impute(validation)
test_imputed = impute(test)

In [13]:
train_imputed[scanners] = normalize(train_imputed[scanners] + 100)
validation_imputed[scanners] = normalize(validation_imputed[scanners] + 100)
test_imputed[scanners] = normalize(test_imputed[scanners] + 100)

In [14]:
X_train, y_train = train_imputed[scanners].copy(), train_imputed['location'].copy()
X_validation, y_validation = validation_imputed[scanners].copy(), validation_imputed['location'].copy()
X_test, y_test = test_imputed[scanners].copy(), test_imputed['location'].copy()

train = train_imputed[scanners + ['location']].copy()
validation = validation_imputed[scanners + ['location']].copy()
test = test_imputed[scanners + ['location']].copy()

In [15]:
pca = PCA(n_components=17)
pca_results = pca.fit_transform(train[scanners])
pca.explained_variance_ratio_

array([0.56052368, 0.09860533, 0.08797432, 0.04614246, 0.02734353,
       0.02661154, 0.02259243, 0.02101593, 0.01844216, 0.01682019,
       0.01543092, 0.01350322, 0.01306483, 0.01227793, 0.00929003,
       0.00818713, 0.00217437])